In [1]:
#import libraries 
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#converting strings into numbers size,

def convert(value):
    if value:
        multiple = 1
        if value.endswith("M"):
            multiple = 1000000
            value = value[0:len(value)-1]
        return int(float(value) * multiple)
    else:
        return 0

In [3]:
#reading google play store data and reviews 

ET_Googleplayrev = "data/googleplaystore_user_reviews.csv"

Googleplayrev_df = pd.read_csv(ET_Googleplayrev)


ET_Googleplay = "data/googleplaystore.csv"

Googleplay_df = pd.read_csv(ET_Googleplay)



#data cleaning for google play store and reviews 
# lets drops rows from data frame where we have naan 

#google play reviews cleaning 

Googleplayrev_df = Googleplayrev_df.dropna(subset={'Translated_Review'})
Googleplayrev_df = Googleplayrev_df.dropna(subset={'Sentiment'})
Googleplayrev_df = Googleplayrev_df.dropna(subset={'Sentiment_Polarity'})
Googleplayrev_df = Googleplayrev_df.dropna(subset={'Sentiment_Subjectivity'})

# Google app cleaning 

Googleplay_df = Googleplay_df.dropna(subset={'Rating', 'Reviews','Size', 'Installs', 'Price'})


#google play cleaning 

#strings into data values 

#Googleplay_df["Size"]=Googleplay_df["Size"].replace({"M":"*1e6"}, regex=True).astype(int)

  
                                                         
#drop type column
Googleplay_df = Googleplay_df.drop(columns='Type')
                                                     
#dropping m from size column 

Googleplay_df['Size'] = Googleplay_df['Size'].str.replace('M', "") 
Googleplay_df['Size'] = Googleplay_df['Size'].str.replace('k', "") 
#dropping the + from installs 

Googleplay_df['Installs'] = Googleplay_df['Installs'].str.replace('+', "") 


#removing "varies with device from size, current version and andriod version"
Googleplay_df = Googleplay_df[Googleplay_df["Size"].str.contains("Varies with device")== False]
Googleplay_df = Googleplay_df[Googleplay_df["Current Ver"].str.contains("Varies with device")== False]
Googleplay_df = Googleplay_df[Googleplay_df["Android Ver"].str.contains("Varies with device")== False]


#changing the date format on google play dataframe 

Googleplay_df['Last Updated'] = pd.to_datetime(Googleplay_df['Last Updated'],format ='%B %d, %Y', errors = 'coerce' )
#Googleplay_df['Last Updated']= pd.to_datetime(Googleplay_df[['Last Updated']])

Googleplay_df

,App,Category,Rating,Reviews,Size,Installs,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19,"10,000",0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14,"500,000",0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,"5,000,000",0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,"100,000",0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6,"50,000",0,Everyone,Art & Design,2017-03-26,1.0,2.3 and up
...,...,...,...,...,...,...,...,...,...,...,...,...
10832,FR Tides,WEATHER,3.8,1195,582,"100,000",0,Everyone,Weather,2014-02-16,6.0,2.1 and up
10833,Chemin (fr),BOOKS_AND_REFERENCE,4.8,44,619,"1,000",0,Everyone,Books & Reference,2014-03-23,0.8,2.2 and up
10834,FR Calculator,FAMILY,4.0,7,2.6,500,0,Everyone,Education,2017-06-18,1.0.0,4.1 and up
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53,"5,000",0,Everyone,Education,2017-07-25,1.48,4.1 and up


In [4]:
Googleplayrev_df

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.000000,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.250000,0.288462
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.400000,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.000000,0.300000
5,10 Best Foods for You,Best way,Positive,1.000000,0.300000
...,...,...,...,...,...
64222,Housing-Real Estate & Property,Most ads older many agents ..not much owner po...,Positive,0.173333,0.486667
64223,Housing-Real Estate & Property,"If photos posted portal load, fit purpose. I'm...",Positive,0.225000,0.447222
64226,Housing-Real Estate & Property,"Dumb app, I wanted post property rent give opt...",Negative,-0.287500,0.250000
64227,Housing-Real Estate & Property,I property business got link SMS happy perform...,Positive,0.800000,1.000000


In [5]:
protocol = 'postgresql'
username = <username>
password = <password>
host = 'localhost'
port = 5444
database_name = 'project_two_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [6]:
Googleplay_df.to_sql(name='google_play_apps', con=engine, if_exists='append', index=False)

In [7]:
Googleplayrev_df.to_sql(name='google_play_apps_revs', con=engine, if_exists='append', index=False)